In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from torchsummary import summary
import os
import time

from dataset import HackathonDataset
from convnet import ConvNet
from resnet import ResNet

from config import DATA_DIR, DEVICE, USE_RAW

In [2]:
class Ensemble:
    
    def __init__(self, Model, device, n_estimators):
        self.Model = Model
        self.instances = [self.Model(device) for i in range(n_estimators)]
    
    def fit(self, train_dataloader, test_dataloader, n_epochs, print_frequency):
        for it, instance in enumerate(self.instances):
            print(f"\n=== Training instance {it+1}/{len(self.instances)} ===\n")
            instance.fit(train_dataloader, test_dataloader, n_epochs, print_frequency)
    
    def predict(self, dataloader):
        predictions = [instance.predict(dataloader) for instance in self.instances]
        return np.mean(predictions, axis=0)

In [6]:
n_epochs = 1
n_estimators = 1
print_frequency = 10
batch_size = 4  # High batch size often happen to not converge... So we use small batches, even if slower
pred_batch_size = 128  # There is no problem of convergence for training batch size

In [7]:
#========================NOTE============================
# We often have to reset the model, because it won't converge. I don't know why, but it is useful to know
# If the training loss is stuck around 22 and the validation loss is stuck around 10,
# reset the model by running this cell again, and relaunch training
#========================END Of NOTE=====================

dataset = HackathonDataset(DATA_DIR + 'mixed_train.csv', DATA_DIR, USE_RAW)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count() - 2)
val_dataset = HackathonDataset(DATA_DIR + 'mixed_validation.csv', DATA_DIR, USE_RAW)
val_dataloader = DataLoader(val_dataset, batch_size=pred_batch_size, shuffle=False, num_workers=os.cpu_count() - 2)
model = Ensemble(ResNet, DEVICE, n_estimators)

In [8]:
model.fit(dataloader, val_dataloader, n_epochs, print_frequency)


=== Training instance 1/1 ===

Epoch 1/1
Number of batches viewed : 1423
Current training loss : 17.487399111142565
Current validation loss : 9.324251967152273
Number of batches viewed : 2847
Current training loss : 9.348210691903414
Current validation loss : 10.461772043873944
Number of batches viewed : 4271
Current training loss : 8.8463914890805
Current validation loss : 10.235857618136668
Number of batches viewed : 5695
Current training loss : 8.437118867809852
Current validation loss : 11.018762321922722
Number of batches viewed : 7119
Current training loss : 8.300056809203678
Current validation loss : 9.84908896168386
Number of batches viewed : 8543
Current training loss : 8.268764884954088
Current validation loss : 10.549730225810855
Number of batches viewed : 9967
Current training loss : 8.171401215654411
Current validation loss : 10.3355223513025
Number of batches viewed : 11391
Current training loss : 8.178669347187107
Current validation loss : 10.631962212990588
Number of b

# Evaluation on Test Data

In [9]:
test_dataset = HackathonDataset(DATA_DIR + 'mixed_test.csv', DATA_DIR, USE_RAW)
test_dataloader = DataLoader(test_dataset, batch_size=pred_batch_size, shuffle=False, num_workers=os.cpu_count() - 2)

In [10]:
image_file_names = []
for val in test_dataloader:
    image_file_names += val['image_file_name']

predictions = model.predict(test_dataloader)
kaggle_df = pd.DataFrame({'image_id': image_file_names,
                          'predicted_z': predictions})

In [11]:
kaggle_df.to_csv('predictions/prediction-' + datetime.now().strftime("%d-%m-%y:%H-%M") + '.csv', index=False)